In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import logging

import pandas as pd
import numpy as np
from hdbscan import HDBSCAN
from sklearn.metrics import pairwise_distances
import plotly.express as px

from rlabs.config import RLabsConfig
from rlabs.data_access import DataClass
from rlabs.model import Model
from rlabs.utils.constants import (
    DATA_CSV,
    CITY,
    STATE,
    COUNTRY,
    LONGITUDE,
    LATITUDE,
    POPULATION,
    MSA,
    USA,
    CLUSTER,
    R
)

pd.set_option('display.max_colwidth', 0)
PARENT_PATH = Path(os.getcwd()).parent.absolute()
FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
config = RLabsConfig()
config.CURRENT_PATH = PARENT_PATH
# For accessing data
data = DataClass(config)
# Model for generating clusters
model = Model(config)

In [4]:
# Load data
df = pd.read_csv(os.path.join(data.data_path, DATA_CSV), index_col=0)
# Keep only US cities
df = df[df[COUNTRY].isin([USA])].copy()
df.reset_index(drop=True, inplace=True)
df.shape

(7484, 7)

In [5]:
X = df[[LONGITUDE, LATITUDE]].to_numpy()
X_radians = np.radians(X)
D_pairwise = pairwise_distances(X_radians, X_radians, metric="haversine")
D_pairwise *= R  # Multiply by Earth radius to get miles
D_pairwise.shape

(7484, 7484)

In [6]:
clusters = HDBSCAN(
    cluster_selection_epsilon=3.0,
    min_cluster_size=2,
    min_samples=1,
    metric="precomputed",
    algorithm="best",
    cluster_selection_method="eom"
).fit(D_pairwise)

df[CLUSTER] = clusters.labels_

In [7]:
print(f"# clusters: {df[CLUSTER].nunique()-1}")

# clusters: 1319


In [8]:
t = df.dropna().copy()
print(f'Number of non-metropolitan cities:\n{t[MSA].str.contains("NON").sum()}')
print(f'\nNumber of unclustered samples (-1 cluster, noise):\n{t[CLUSTER].isin([-1]).sum()}')

numerator = t[t[CLUSTER].isin([-1])][MSA].str.contains("NON").sum()
denominator = t[t[CLUSTER].isin([-1])].shape[0]
print(f'\nHow many non-metropolitan cities make up the -1 cluster? (Precision):\n{(numerator/denominator) * 100:.4}%')

numerator = t[t[MSA].str.contains("NON")][CLUSTER].isin([-1]).sum()
denominator = t[t[MSA].str.contains("NON")].shape[0]
print(f'\nWhat proportion of non-metropolitan cities were detected as noise and mapped to -1 cluster? (Recall):\n{(numerator/denominator) * 100:.4}%')

Number of non-metropolitan cities:
1769

Number of unclustered samples (-1 cluster, noise):
610

How many non-metropolitan cities make up the -1 cluster? (Precision):
46.39%

What proportion of non-metropolitan cities were detected as noise and mapped to -1 cluster? (Recall):
16.0%


In [9]:
df_merged = model.merge(
    df=df.copy(),
    D=D_pairwise,
    distance_threshold=10.0,
    population_threshold=50000,
    max_iter=2000,
    verbose=100
)

2022-12-26 19:46:02,292 - rlabs.model.core - INFO - Merging clusters ...
2022-12-26 19:46:02,293 - rlabs.model.core - INFO - population_threshold: 50000
2022-12-26 19:46:02,293 - rlabs.model.core - INFO - distance_threshold: 10.0
2022-12-26 19:46:02,294 - rlabs.model.core - INFO - max_iter: 2000
2022-12-26 19:49:58,570 - rlabs.model.core - INFO - (100/2000) 1219 clusters left
2022-12-26 19:54:09,891 - rlabs.model.core - INFO - (200/2000) 1119 clusters left
2022-12-26 19:59:04,024 - rlabs.model.core - INFO - (300/2000) 1019 clusters left
2022-12-26 20:04:33,742 - rlabs.model.core - INFO - (400/2000) 919 clusters left
2022-12-26 20:11:14,802 - rlabs.model.core - INFO - (500/2000) 819 clusters left
2022-12-26 20:19:19,571 - rlabs.model.core - INFO - (600/2000) 719 clusters left
2022-12-26 20:27:06,309 - rlabs.model.core - INFO - (700/2000) 619 clusters left
2022-12-26 20:37:14,890 - rlabs.model.core - INFO - (800/2000) 519 clusters left
2022-12-26 20:47:26,838 - rlabs.model.core - INFO - 

In [16]:
t = df_merged.dropna().copy()
print(f'\n# MSA {t[~t[MSA].str.contains("NON")][MSA].nunique()}')
print(f"\n# clusters after merge: {df_merged[CLUSTER].nunique()-1}")


# MSA 362

# clusters after merge: 360


In [17]:
t = df_merged.dropna().copy()
print(f'Number of non-metropolitan cities:\n{t[MSA].str.contains("NON").sum()}')
print(f'\nNumber of unclustered samples (-1 cluster, noise):\n{t[CLUSTER].isin([-1]).sum()}')

numerator = t[t[CLUSTER].isin([-1])][MSA].str.contains("NON").sum()
denominator = t[t[CLUSTER].isin([-1])].shape[0]
print(f'\nHow many non-metropolitan cities make up the -1 cluster? (Precision):\n{(numerator/denominator) * 100:.4}%')

numerator = t[t[MSA].str.contains("NON")][CLUSTER].isin([-1]).sum()
denominator = t[t[MSA].str.contains("NON")].shape[0]
print(f'\nWhat proportion of non-metropolitan cities were detected as noise and mapped to -1 cluster? (Recall):\n{(numerator/denominator) * 100:.4}%')

Number of non-metropolitan cities:
1769

Number of unclustered samples (-1 cluster, noise):
1225

How many non-metropolitan cities make up the -1 cluster? (Precision):
58.04%

What proportion of non-metropolitan cities were detected as noise and mapped to -1 cluster? (Recall):
40.19%


Save dataframe

In [18]:
save_to = os.path.join(data.reports_path, "us_cities.csv")
df_merged.to_csv(save_to, index=False)

##### NY

In [19]:
state_name = "new york"

fig = px.scatter(
    df_merged[df_merged[STATE].isin([state_name])],
    y=LATITUDE,
    x=LONGITUDE,
    color=CLUSTER,
    hover_data=[CITY, STATE, MSA, POPULATION],
    height=400,
    width=800,
    title=f"{state_name}"
)
fig.update_layout(uniformtext_mode="hide", showlegend=False)
fig.show()

##### CA

In [20]:
state_name = "california"

fig = px.scatter(
    df_merged[df_merged[STATE].isin([state_name])],
    y=LATITUDE,
    x=LONGITUDE,
    color=CLUSTER,
    hover_data=[CITY, STATE, MSA, POPULATION],
    height=400,
    width=800,
    title=f"{state_name}"
)
fig.update_layout(uniformtext_mode="hide", showlegend=False)
fig.show()

##### WV

In [21]:
state_name = "west virginia"

fig = px.scatter(
    df_merged[df_merged[STATE].isin([state_name])],
    y=LATITUDE,
    x=LONGITUDE,
    color=CLUSTER,
    hover_data=[CITY, STATE, MSA, POPULATION],
    height=400,
    width=800,
    title=f"{state_name}"
)
fig.update_layout(uniformtext_mode="hide", showlegend=False)
fig.show()

Note the change from `analysis.ipynb` on `west virginia`.